<a href="https://colab.research.google.com/github/PhuongHa159/Python-E-Wallet-Data-Analytics/blob/main/H%C3%80_C%C3%81C_PH%C6%AF%E1%BB%A2NG_E_WALLET_DATA_ANALYTICS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving transactions.csv to transactions.csv
Saving product.csv to product.csv
Saving payment_report.csv to payment_report.csv


In [ ]:
import pandas as pd
  #import file to DataFrame
transactions = pd.read_csv("transactions.csv")
product = pd.read_csv("product.csv")
payment_report = pd.read_csv("payment_report.csv")

  #Df payment_enriched (Merge payment_report.csv with product.csv)
payment_enriched = pd.merge_ordered(payment_report,product,on = 'product_id',how="left")

df_pay=pd.DataFrame(payment_enriched)
df_trans=pd.DataFrame(transactions)

In [ ]:
df_pay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   report_month   919 non-null    object
 1   payment_group  919 non-null    object
 2   product_id     919 non-null    int64 
 3   source_id      919 non-null    int64 
 4   volume         919 non-null    int64 
 5   category       897 non-null    object
 6   team_own       897 non-null    object
dtypes: int64(3), object(4)
memory usage: 50.4+ KB


In [ ]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324002 entries, 0 to 1324001
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   transaction_id  1324002 non-null  int64  
 1   merchant_id     1324002 non-null  int64  
 2   volume          1324002 non-null  int64  
 3   transType       1324002 non-null  int64  
 4   transStatus     1324002 non-null  int64  
 5   sender_id       1274943 non-null  float64
 6   receiver_id     1159207 non-null  float64
 7   extra_info      6095 non-null     object 
 8   timeStamp       1324002 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 90.9+ MB


#PART 1: EDA
##Check each column: missing data? duplicates? incorrect data types?
###Payment_enriched

In [ ]:
    # check missing data
print(df_pay.isna().sum())

report_month      0
payment_group     0
product_id        0
source_id         0
volume            0
category         22
team_own         22
dtype: int64


In [ ]:
#dropping missing value
threshold_pay=len(df_pay) *0.05
cols_to_drop=df_pay.columns[(df_pay.isna().sum() > 0) & (df_pay.isna().sum() < threshold_pay)]
df_pay.dropna(subset=cols_to_drop,inplace=True)
print(df_pay.isna().sum())

report_month     0
payment_group    0
product_id       0
source_id        0
volume           0
category         0
team_own         0
dtype: int64


In [ ]:
# duplicate?
duplicates = df_pay.duplicated().sum()
print("Duplicate in Payment enriched:",duplicates)

Duplicate in Payment enriched: 0


In [ ]:
#incorrect data types?
print(df_pay.dtypes)
df_pay["report_month"] = pd.to_datetime(df_pay['report_month'], format='%Y-%m')
print(df_pay.dtypes)

report_month     datetime64[ns]
payment_group            object
product_id                int64
source_id                 int64
volume                    int64
category                 object
team_own                 object
dtype: object
report_month     datetime64[ns]
payment_group            object
product_id                int64
source_id                 int64
volume                    int64
category                 object
team_own                 object
dtype: object


In [ ]:
    #check outliner
def find_outliers_iqr(df_pay):
  outliers = pd.DataFrame()
for column in df_pay.select_dtypes(include=['int']).columns:
        Q1 = df_pay[column].quantile(0.25)
        Q3 = df_pay[column].quantile(0.75)
        IQR = Q3 - Q1
      # Lọc các giá trị bất thường và thêm vào DataFrame kết quả
        outliers_column = df_pay[(df_pay[column] < (Q1 - 1.5 * IQR)) | (df_pay[column] > (Q3 + 1.5 * IQR))]
      # Gọi hàm để tìm các giá trị bất thường
outliers = find_outliers_iqr(df_pay)
print("Outlier in Payment_enriched:",outliers)

Outlier in Payment_enriched: None


###transactions

In [ ]:
  #check missing value
print(df_trans.isna().sum())
threshold_trans=len(df_trans)*0.05

transaction_id          0
merchant_id             0
volume                  0
transType               0
transStatus             0
sender_id           49059
receiver_id        164795
extra_info        1317907
timeStamp               0
dtype: int64


In [ ]:
#dropping missing value
transactions_drop=df_trans.columns[(df_trans.isna().sum() > 0) & (df_trans.isna().sum() <= threshold_trans)]
df_trans.dropna(subset=transactions_drop,inplace=True)
df_trans.drop(columns=['extra_info'], inplace=True)
print(df_trans.isna().sum())


transaction_id         0
merchant_id            0
volume                 0
transType              0
transStatus            0
sender_id              0
receiver_id       162208
timeStamp              0
dtype: int64


In [ ]:
  #Imputing data
cols_to_impute=df_trans.columns[df_trans.isna().sum() > threshold_trans]
for col in cols_to_impute:
  df_trans[col] = df_trans[col].fillna(df_trans[col].mode()[0])
print(df_trans.isna().sum())

transaction_id    0
merchant_id       0
volume            0
transType         0
transStatus       0
sender_id         0
receiver_id       0
timeStamp         0
dtype: int64


In [ ]:
# duplicate?
duplicates = df_trans.duplicated().sum()
print("Duplicates in transactions:",duplicates)
df_trans.drop_duplicates(keep='last')

Duplicates in transactions: 27


,transaction_id,merchant_id,volume,transType,transStatus,sender_id,receiver_id,timeStamp
0,3002692434,5,100000,24,1,10199794.0,199794.0,1682932054455
1,3002692437,305,20000,2,1,14022211.0,14022211.0,1682932054912
3,3002680710,2270,1500000,2,1,10059206.0,59206.0,1682932055622
4,3002680713,2275,90000,2,1,10004711.0,4711.0,1682932056197
5,3002680716,2270,200000,2,1,10162232.0,162232.0,1682932056816
...,...,...,...,...,...,...,...,...
1323997,3003723030,305,20000,2,1,24524311.0,10563903.0,1683035672634
1323998,3003723033,2270,100000,2,1,10277242.0,277242.0,1683035672876
1323999,3003723036,2270,100000,2,1,10144599.0,144599.0,1683035672892
1324000,3003723039,5,400,22,1,10028007.0,21013762.0,1683035672896


In [ ]:
    #incorrect data types?
print(df_trans.dtypes)
df_trans["sender_id"] = df_trans["sender_id"].astype(int)
df_trans["receiver_id"] = df_trans["receiver_id"].astype(int)
print(df_trans.dtypes)

transaction_id      int64
merchant_id         int64
volume              int64
transType           int64
transStatus         int64
sender_id         float64
receiver_id       float64
timeStamp           int64
dtype: object
transaction_id    int64
merchant_id       int64
volume            int64
transType         int64
transStatus       int64
sender_id         int64
receiver_id       int64
timeStamp         int64
dtype: object


In [ ]:
    #check outliner
def find_outliers_iqr(df_trans):
  outliers = pd.DataFrame()
for column in df_trans.select_dtypes(include=['int']).columns:
        Q1 = df_trans[column].quantile(0.25)
        Q3 = df_trans[column].quantile(0.75)
        IQR = Q3 - Q1
      # Filter outlier and add to Dataframe
        outliers_column = df_trans[(df_trans[column] < (Q1 - 1.5 * IQR)) | (df_trans[column] > (Q3 + 1.5 * IQR))]
      # Find outlier
outliers = find_outliers_iqr(df_trans)
print("Outlier in transactions:",outliers)

Outlier in transactions: None


#Part II: Data Wrangling

In [ ]:
# 1. Top 3 product_ids with the highest volume
top_three_product=payment_report.groupby("product_id")["volume"].sum().nlargest(3)

print("Top 3 product_ids with the highest volume:", top_three_product)


Top 3 product_ids with the highest volume: product_id
1976    61797583647
429     14667676567
372     13713658515
Name: volume, dtype: int64


In [ ]:
# 2. Given that 1 product_id is only owed by 1 team, are there any abnormal products against this rule?

  # Find 1 product_id is only owed by 1 team
abnormal_product = product.groupby("product_id")["team_own"].nunique()

  # Find abnormal products against this rule
abnormal_product = abnormal_product[abnormal_product > 1]

print("Abnormal products against this rule:",abnormal_product)

Abnormal products against this rule: Series([], Name: team_own, dtype: int64)


In [ ]:
# 3. Find the team has had the lowest performance (lowest volume) since Q2.2023. Find the category that contributes the least to that team.
  # Find data since Q2.2023
payment_q2_2023 = payment_enriched[payment_enriched["report_month"].isin(['2023-04', '2023-05', '2023-06'])]

  # Find the team has had the lowest performance (lowest volume) since Q2.2023
lowest_team=payment_q2_2023.groupby("team_own")["volume"].sum().idxmin()

  # Find the category that contributes the least to that team.
team_low = payment_q2_2023[payment_q2_2023["team_own"] == lowest_team]
least_category= team_low.groupby("category")["volume"].sum().idxmin()

print("The team has had the lowest volume since Q2.2023:", lowest_team)
print("The category that contributes the least:", least_category)

The team has had the lowest volume since Q2.2023: APS
The category that contributes the least: PXXXXXE


In [ ]:
# 4. Find the contribution of source_ids of refund transactions (payment_group = ‘refund’), what is the source_id with the highest contribution?

  #Find refund transactions
refund_transactions=payment_report[payment_report["payment_group"]=="refund"]

  #Find the contribution of source_ids of refund transactions
source_id_contribution=refund_transactions.groupby("source_id")["volume"].sum()

  #Find the source_id with the highest contribution
highest_contribution=source_id_contribution.idxmax()

print("The contribution of source_ids of refund transactions:",source_id_contribution)
print("The source_id with the highest contribution:",highest_contribution)


The contribution of source_ids of refund transactions: source_id
37     9151069226
38    36527454759
39    16119059662
Name: volume, dtype: int64
The source_id with the highest contribution: 38


In [ ]:
# 5. Define type of transactions (‘transaction_type’) for each row
def type_transactions(row):
  if row["transType"]==2 :
    if row["merchant_id"] == 1205 :
      return "Bank Transfer Transaction"
    elif row["merchant_id"] == 2260 :
      return "Withdraw Money Transaction"
    elif row["merchant_id"] == 2270 :
      return "Top Up Money Transaction"
    else:
      return "Payment Transaction"
  elif row["transType"] == 8 :
    if row["merchant_id"] == 2250 :
      return "Transfer Money Transaction"
    else :
      return "Split Bill Transaction"
  else :
    return "Invalid transactions"

    #Apply transaction_type in transactions
transactions['transaction_type'] = transactions.apply(type_transactions, axis=1)
print("Transaction_type in transactions:",transactions.head())

Transaction_type in transactions:    transaction_id  merchant_id   volume  transType  transStatus   sender_id  \
0      3002692434            5   100000         24            1  10199794.0   
1      3002692437          305    20000          2            1  14022211.0   
2      3001960110         7255    48605         22            1         NaN   
3      3002680710         2270  1500000          2            1  10059206.0   
4      3002680713         2275    90000          2            1  10004711.0   

   receiver_id extra_info      timeStamp          transaction_type  
0     199794.0        NaN  1682932054455      Invalid transactions  
1   14022211.0        NaN  1682932054912       Payment Transaction  
2   10530940.0        NaN  1682932055000      Invalid transactions  
3      59206.0        NaN  1682932055622  Top Up Money Transaction  
4       4711.0        NaN  1682932056197       Payment Transaction  


In [ ]:
# 6. Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers.

# Find valid_transaction(excluding invalid transactions)
valid_transaction=transactions[transactions['transaction_type'] != 'Invalid transactions']

# Find the number of transactions, volume, senders and receivers
valid_transaction_describe = valid_transaction.groupby("transaction_type").agg(
    num_transaction=("transaction_id","count"),
    sum_volume=("volume","sum"),
    num_sender=("sender_id","nunique"),
    num_receiver=("receiver_id","nunique")
)
print("The number of transactions, volume, senders and receivers:",valid_transaction_describe)

The number of transactions, volume, senders and receivers:                             num_transaction    sum_volume  num_sender  \
transaction_type                                                        
Bank Transfer Transaction             37879   50605806190       23156   
Payment Transaction                  398677   71851515181      139583   
Split Bill Transaction                 1376       4901464        1323   
Top Up Money Transaction             290502  108606478829      110409   
Transfer Money Transaction           341177   37033171492       39021   
Withdraw Money Transaction            33725   23418181420       24814   

                            num_receiver  
transaction_type                          
Bank Transfer Transaction           9271  
Payment Transaction               113298  
Split Bill Transaction               572  
Top Up Money Transaction          110409  
Transfer Money Transaction         34585  
Withdraw Money Transaction         24814  
